<a href="https://colab.research.google.com/github/DivyanshSareen/Movie_Scrape/blob/master/scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url_hi = 'https://timesofindia.indiatimes.com/entertainment/hindi/movie-reviews?curpg='  # curpg!
url_en = 'https://timesofindia.indiatimes.com/entertainment/english/movie-reviews?curpg='

In [ ]:
def get_urls(url, n):   
    pg_no = 1
    temp = url
    url_li = []
    name_li = []
    error_curpg = []
    while True:
        try:
            print(url+str(pg_no))
            page = requests.get(url+str(pg_no))
            soup = BeautifulSoup(page.text, 'html.parser')
            link_data = soup.find_all("div", class_="FIL_left")
            name_data = soup.find_all("div", class_='FIL_right')
            for ele in link_data:
                a = ele.find()
                a_href = a['href']
                a_split = a_href.split("/")
                url_li.append((a_split[4],a_split[6]))
                # print(a['href'])
            for ele in name_data:
                a = ele.find('h3')
                name_li.append(a.text)
        except e:
            error_curpg.append(url+str(pg_no))
        if len(link_data) == 0:
            break
        if pg_no == n:
            break
        pg_no = pg_no+1
        url = temp
    print("failed get the following links:", error_curpg)
    return url_li, name_li

In [ ]:
x,y = get_urls(url_hi, 50)

https://timesofindia.indiatimes.com/entertainment/hindi/movie-reviews?curpg=1
https://timesofindia.indiatimes.com/entertainment/hindi/movie-reviews?curpg=2
https://timesofindia.indiatimes.com/entertainment/hindi/movie-reviews?curpg=3
https://timesofindia.indiatimes.com/entertainment/hindi/movie-reviews?curpg=4
https://timesofindia.indiatimes.com/entertainment/hindi/movie-reviews?curpg=5
https://timesofindia.indiatimes.com/entertainment/hindi/movie-reviews?curpg=6
https://timesofindia.indiatimes.com/entertainment/hindi/movie-reviews?curpg=7
https://timesofindia.indiatimes.com/entertainment/hindi/movie-reviews?curpg=8
https://timesofindia.indiatimes.com/entertainment/hindi/movie-reviews?curpg=9
https://timesofindia.indiatimes.com/entertainment/hindi/movie-reviews?curpg=10
https://timesofindia.indiatimes.com/entertainment/hindi/movie-reviews?curpg=11
https://timesofindia.indiatimes.com/entertainment/hindi/movie-reviews?curpg=12
https://timesofindia.indiatimes.com/entertainment/hindi/movie

In [ ]:
class Movie:
    error_links = []
    def __init__(self, id, lang):
        self.url = self.generate_url(id, lang)
        try:
            self.soup = self.generate_soup(self.url)
        except:
            error_links.append(id)
        self.date = self.get_date(self.soup)
        self.lang = self.get_language(self.soup)
        self.dur = self.get_duration(self.soup)
        self.genre = self.get_genre(self.soup)
        self.direc = self.get_director(self.soup)
        self.act = self.get_actor(self.soup)
        self.c_rate = self.get_crit_rating(self.soup)
        self.u_rate = self.get_user_rating(self.soup)
        self.desc = self.get_synopsis(self.soup)
        self.poster = self.get_poster(self.soup)

    def generate_url(self, id, lang):
        try:
            if lang == "hi":
                return "https://timesofindia.indiatimes.com/entertainment/hindi/movie-reviews/"+id[0]+"/movie-review/"+id[1]
            elif lang == "en":
                return "https://timesofindia.indiatimes.com/entertainment/english/movie-reviews/"+id[0]+"/movie-review/"+id[1]
        except:
            return None
    def generate_soup(self, url):
        try:
            page = requests.get(url)
            soup = BeautifulSoup(page.text, 'html.parser')
            return soup
        except: 
            return None
    def get_date(self, soup):
        try:
            temp = soup.find_all('div', class_="md_info")
            return temp[0].text
        except: 
            return None
    def get_language(self, soup):
        try:
            temp = soup.find_all('div', class_="md_info")
            temp1 = temp[1].find_all('a')
            return [ele.text for ele in temp1]
        except: 
            return None
    def get_duration(self, soup):
        try:
            temp = soup.find_all('div', class_="md_info")
            return temp[2].text.strip()
        except: 
            return None
    def get_genre(self, soup):
        try:
            temp = soup.find_all('div', class_="md_info")
            return temp[3].text
        except: 
            return None
    def get_crit_rating(slef, soup):
        try:
            temp = soup.find('span', class_="cricrating")
            return temp.text
        except:
            return None
    def get_user_rating(self, soup):
        try:
            temp = soup.find_all('div', class_=['nbstar'])
            temp1 = temp[1].find('span',class_="rate_count")
            return temp1.text.split("/")[0]
        except:
            return None
    def get_director(self, soup):
        try:
            temp = soup.find('div', class_='castcrew_scroller')
            return temp.find('li').text.replace("Director","")
        except:
            return None
    def get_actor(self, soup):
        try:
            temp = soup.find('div', class_='castcrew_scroller')
            temp1 = temp.find_all('li')
            temp2 = []
            for i in range(1,4):
                temp2.append(temp1[i].text.replace("Actor",""))
            return temp2
        except:
            return None
    def get_synopsis(self, soup):
        try:
            temp = soup.find("div", class_="lesssynocont htfix")
            return temp.text
        except:
            return None
    def get_poster(self, soup):
        try:
            temp = soup.find('div',class_="movie_poster").find('img')
            return temp['src']
        except:
            return None

In [ ]:
df_en = pd.DataFrame({
    'name':[],
    'date':[],
    'lang':[],
    'dur':[],
    'genre':[],
    'direc':[],
    'act':[],
    'c_rate':[],
    'u_rate':[],
    'desc':[],
    'poster':[]
})

for i in range(len(x)):
    movie = Movie(x[i], "hi")
    df_en = df_en.append({
    'name': y[i],
    'date': movie.date,
    'lang': movie.lang,
    'dur' : movie.dur,
    'genre' : movie.genre,
    'direc':movie.direc,
    'act':movie.act,
    'c_rate':movie.c_rate,
    'u_rate': movie.u_rate,
    'desc': movie.desc,
    'poster': movie.poster}, ignore_index = True)
    print(y[i], movie.date, movie.lang, movie.dur, movie.genre,movie.direc, movie.act, movie.c_rate, movie.u_rate, movie.desc, movie.poster)
    del movie

Radhe 13 May, 2021 ['Hindi'] 2 hrs 15 mins  Action Thriller Crime Prabhudeva ['Salman Khan', 'Disha Patani', 'Randeep Hooda'] 3.0 3.0 This is definitely not the most wanted Bhai film, but the die-hard Salman fans might just find a reason to cheer him on, yet again. https://static.toiimg.com/thumb/msid-80380429,width-219,height-317,imgsize-37612/80380429.jpg
99 Songs 16 Apr, 2021 ['Hindi', 'Tamil', 'Telugu'] 2 hrs 13 mins  Romance Music Vishwesh Krishnamoorthy ['Ehan Bhat', 'Edilsy Vargas', 'Lisa Ray'] 3.0 3.3 A budding musician takes up the challenge to compose one song that can change the world, to unite with his girlfriend.  https://static.toiimg.com/thumb/msid-68883903,width-219,height-317,imgsize-68551/68883903.jpg
Bansuri: The Flute 16 Apr, 2021 ['Hindi'] 1 hr 48 mins Drama Hari Viswanathan ['Rituparna Sengupta', 'Anurag Kashyap', 'Upendra Limaye'] 2.5 2.6 In a nutshell, ‘Bansuri’ has its heart at the right place, however, it fails to leave a lasting impact on the audience. Truth 

In [ ]:
df_en.head()
df_en.to_excel("movie_hindi.xlsx",index = False)